In [1]:
!python -m pip install --upgrade pip

In [2]:
import os
import time
import codecs

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager


In [3]:
USERNAME = "amelia99"
PASSWORD = os.environ.get('password') 
if PASSWORD is None:
    PASSWORD = input("Please enter your password: ")

In [4]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
MAIN_URL = "https://farmrpg.com/"
driver.get(MAIN_URL)

In [150]:
location_urls = pd.read_csv("location_urls.csv")
location_urls

,name,url
0,home,https://farmrpg.com/
1,farm,https://farmrpg.com/#!/xfarm.php?id=343807
2,explore,https://farmrpg.com/#!/explore.php
3,mount banon,https://farmrpg.com/#!/area.php?id=8
4,forest,https://farmrpg.com/#!/area.php?id=7
5,help,https://farmrpg.com/#!/quests.php
6,inventory,https://farmrpg.com/#!/inventory.php
7,login,https://farmrpg.com/#!/login.php
8,home,https://farmrpg.com/#!/index.php
9,fish,https://farmrpg.com/#!/fish.php


In [151]:
location_xpaths = pd.read_csv("location_xpaths.csv", delimiter=";")
location_xpaths

,name,xpath
0,farm,"//div[@class=""view view-main navbar-through""]/..."
1,inventory,"//div[@class=""view view-main navbar-through""]/..."
2,explore,"//div[@class=""view view-main navbar-through""]/..."
3,fish,"//div[@class=""view view-main navbar-through""]/..."
4,workshop,"//div[@class=""view view-main navbar-through""]/..."
5,town,"//div[@class=""view view-main navbar-through""]/..."


In [7]:
def get_current_location():
    print(driver.current_url)

    try:
        return location_urls[location_urls["url"] == driver.current_url].iloc[0,0]
    except:
        print(f"Location {driver.current_url} not found in locations.csv")
        return None

get_current_location()

https://farmrpg.com/


'home'

In [8]:
def is_refresh_button_available():
    return bool(len(driver.find_elements(By.XPATH, "//i[@aria-label='Refresh Game']")))

In [180]:
def goto(target_location):
    target_location = target_location.lower()
    current_location = get_current_location()
    if current_location != target_location:
        print(f"GOTO:  {current_location} -> {target_location}")
        
        driver.get(MAIN_URL)
        
        if target_location == "home":
            return
        elif target_location in location_xpaths["name"].values.tolist():
            xpath =  location_xpaths.loc[location_xpaths["name"] == target_location,"xpath"].iloc[0]
            print(xpath)
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath)))
            driver.find_element(By.XPATH, xpath).click()
            time.sleep(0.1)
        else:
            print("Sorry, this Location has not been added yet!")

# goto("inventory")
# location_xpaths["name"].values.tolist()

In [10]:
def is_logged_in():
    goto("home")

    if driver.find_element(By.XPATH, "//div[@id='logged_in_username']").get_attribute("innerHTML") == USERNAME:
        return True
    return False

In [11]:
def login_user(username, password):
    if is_logged_in():
        print(f"Already logged in as {username}")
        return

    login_div = driver.find_element(By.XPATH, "//div[contains(text(), 'Login')]")
    login_div.click()

    time.sleep(1)

    username_input = driver.find_element(By.XPATH, "//input[@name='username']")
    username_input.send_keys(username)

    password_input = driver.find_element(By.XPATH, "//input[@name='password']")
    password_input.send_keys(password)

    login_button = driver.find_element(By.XPATH, "//input[@value='Login']")
    login_button.click()

    time.sleep(1)

    if is_logged_in():
        print(f"Successfully logged in as {username}")
    else:
        login_user(username, password)

login_user(USERNAME, PASSWORD)


https://farmrpg.com/
https://farmrpg.com/index.php
Location https://farmrpg.com/index.php not found in locations.csv
GOTO:  None -> home
Successfully logged in as amelia99


In [13]:
def set_farm_url():
    goto("home")
    
    farm_id = driver.find_element(By.XPATH, "//a[contains(@href, 'xfarm.php?id')]").get_attribute("href").split("id=")[-1]
    location_urls.loc[location_urls["name"] == "farm","url"] = f"https://farmrpg.com/#!/xfarm.php?id={farm_id}"
    print(f"Set Farm URL to {location_urls.loc[location_urls['name'] == 'farm','url'].iloc[0]}")

set_farm_url()
location_urls
    

https://farmrpg.com/
Set Farm URL to https://farmrpg.com/#!/xfarm.php?id=343807


,name,url
0,home,https://farmrpg.com/
1,farm,https://farmrpg.com/#!/xfarm.php?id=343807
2,explore,https://farmrpg.com/#!/explore.php
3,mount banon,https://farmrpg.com/#!/area.php?id=8
4,forest,https://farmrpg.com/#!/area.php?id=7
5,help,https://farmrpg.com/#!/quests.php
6,inventory,https://farmrpg.com/#!/inventory.php
7,login,https://farmrpg.com/#!/login.php
8,home,https://farmrpg.com/#!/index.php
9,fish,https://farmrpg.com/#!/fish.php


In [181]:
def harvest():
    goto("farm")
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Harvest All')]")))
    time.sleep(0.01)
    try:
        driver.find_element(By.XPATH, '//a[contains(text(), "Harvest All")]').click()
    except StaleElementReferenceException:
        driver.refresh()
        driver.find_element(By.XPATH, '//a[contains(text(), "Harvest All")]').click()

harvest()

https://farmrpg.com/#!/town.php
GOTO:  town -> farm
//div[@class="view view-main navbar-through"]//a[contains(@href, 'xfarm.php?id')]


In [182]:
def get_available_crops(crop_select=None):
    goto("farm")
    
    if crop_select is None:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//select[@class='seedid inlineinputlg']")))
        crop_select = Select(driver.find_element(By.XPATH, '//select[@class="seedid inlineinputlg"]'))

    return pd.DataFrame([[o.get_attribute("data-name"), o.get_attribute("data-amt")] for o in crop_select.options], columns=["name", "amount"]).dropna()

get_available_crops()

https://farmrpg.com/#!/xfarm.php?id=343807


,name,amount
1,Cabbage Seeds,43
2,Carrot Seeds,188
3,Cucumber Seeds,341
4,Eggplant Seeds,486
5,Gold Carrot Seeds,15
6,Gold Cucumber Seeds,10
7,Gold Pea Seeds,10
8,Gold Pepper Seeds,10
9,Hops Seeds,150
10,Leek Seeds,341


In [183]:
def plant_crop(crop_name):
    goto("farm")
    
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//select[@class='seedid inlineinputlg']")))
    crop_select = Select(driver.find_element(By.XPATH, '//select[@class="seedid inlineinputlg"]'))

    available_crops = get_available_crops(crop_select=crop_select)

    crop_name = crop_name[0].upper() + crop_name[1:].lower()
    if "seeds" in crop_name.lower():
        crop_name = crop_name.replace("seeds","").replace("Seeds","").replace(" ","")
    crop_name += " Seeds"


    crop_info = available_crops[available_crops.name == crop_name]

    if crop_info.empty:
        print(f"Could not find crop {crop_name} in available crops")
        # TODO: implement buying
        return

    crop_select.select_by_index(crop_info.index.values[0])

    driver.find_element(By.XPATH, '//a[contains(text(), "Plant All")]').click()
    print(f'Planted {crop_name}')
    
plant_crop("carrot")

https://farmrpg.com/#!/xfarm.php?id=343807
https://farmrpg.com/#!/xfarm.php?id=343807
Planted Carrot Seeds


In [184]:
def is_harvest_available():
    goto("home")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'xfarm.php?id')]//span[contains(@class,'ready ready')]")))
    message = driver.find_element(By.XPATH, "//a[contains(@href, 'xfarm.php?id')]//span[contains(@class,'ready ready')]").get_attribute("innerHTML")

    if message == "":
        print("No Crops planted!")
        return False
    elif " Growing" in message:
        print("Crops are still growing!")
        return False
    elif " READY!" in message:
        print(message.replace('<span style="color:#30d611">',"").replace("</span>",""))
        return True

is_harvest_available()
    

https://farmrpg.com/#!/xfarm.php?id=343807
GOTO:  farm -> home
Crops are still growing!


False

In [18]:
def get_explore_locations():
    goto("explore")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")
    return [location.get_attribute("innerHTML").split("<br>")[0] for location in locations]

get_explore_locations()

https://farmrpg.com/
GOTO:  home -> explore
//div[@class="view view-main navbar-through"]//a[@href='explore.php']


['Forest',
 'Small Cave',
 'Small Spring',
 'Haunted House',
 'Highland Hills',
 'Cane Pole Ridge',
 'Misty Forest',
 'Black Rock Canyon',
 'Mount Banon',
 'Ember Lagoon']

In [202]:
def explore(target_location=None, num=5, rate=10, replenish=True):
    if "area" not in driver.current_url:
        goto("explore")
            
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")))
        locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")

        print([loc.get_attribute("innerHTML").split("<br>")[0] for loc in locations])
        
        if target_location is None:
            locations[-1].click()
        else:
            for location in locations:
                if target_location.lower() == location.get_attribute("innerHTML").split("<br>")[0].lower():
                    location.click()
                    break
            else:
                print(f"Could not find location {target_location}")
                return

    time.sleep(0.5)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@id='exploreconsole']")))
    tab = driver.find_element(By.XPATH, "//div[@id='exploreconsole']")

    for i in range(num):
        if "You have run out of stamina and cannot continue. Eat Apples, Drink OJ or fish at the" in tab.get_attribute("innerHTML"):
            print("No stamina left")
            if replenish: 
                replenish_stamina()
                explore(target_location=target_location, num=num-i, rate=rate, replenish=True)
                break
            else:
                break
        tab.click()
        time.sleep(1/rate)


explore("forest", num=100)

https://farmrpg.com/#!/fishing.php?id=2
Location https://farmrpg.com/#!/fishing.php?id=2 not found in locations.csv
GOTO:  None -> explore
//div[@class="view view-main navbar-through"]//a[@href='explore.php']
['Forest', 'Small Cave', 'Small Spring', 'Highland Hills', 'Cane Pole Ridge', 'Misty Forest', 'Black Rock Canyon', 'Mount Banon', 'Ember Lagoon']
No stamina left
https://farmrpg.com/#!/area.php?id=7
GOTO:  forest -> fish
//div[@class="view view-main navbar-through"]//a[@href='fish.php']


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=119.0.6045.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00BE72A3+45731]
	(No symbol) [0x00B72D51]
	(No symbol) [0x00A6880D]
	(No symbol) [0x00A6C3EA]
	(No symbol) [0x00A6DB1A]
	(No symbol) [0x00A6DBC0]
	(No symbol) [0x00A983BD]
	(No symbol) [0x00AB7FDC]
	(No symbol) [0x00A94A4E]
	(No symbol) [0x00AB8254]
	(No symbol) [0x00ACB7A2]
	(No symbol) [0x00AB7DD6]
	(No symbol) [0x00A931F6]
	(No symbol) [0x00A9439D]
	GetHandleVerifier [0x00EF0716+3229462]
	GetHandleVerifier [0x00F384C8+3523784]
	GetHandleVerifier [0x00F3214C+3498316]
	GetHandleVerifier [0x00C71680+611968]
	(No symbol) [0x00B7CCCC]
	(No symbol) [0x00B78DF8]
	(No symbol) [0x00B78F1D]
	(No symbol) [0x00B6B2C7]
	BaseThreadInitThunk [0x761BFCC9+25]
	RtlGetAppContainerNamedObje

In [22]:
def get_fish_locations():
    goto("fish")
        
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'fishing.php')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'fishing.php')]//div[@class='item-title']")
    return [location.get_attribute("innerHTML").split("<br>")[0] for location in locations]

get_fish_locations()

https://farmrpg.com/#!/fish.php


['Farm Pond',
 'Small Pond',
 'Forest Pond',
 'Lake Tempest',
 'Small Island',
 'Crystal River',
 'Emerald Beach',
 'Vast Ocean']

In [25]:
def go_to_fish_location(target_location=None):
    goto("fish")
        
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'fishing')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'fishing')]//div[@class='item-title']")

    if target_location is None:
        locations[-1].click()
        return
    
    for location in locations:
        if target_location.lower() in location.get_attribute("innerHTML").lower():
            location.click()
            return

    print(f"Location {target_location} not found!")

In [198]:
def buy_bait():
    goto("town")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[@href='store.php']")))
    driver.find_element(By.XPATH, "//a[@href='store.php']").click()

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@data-page='store']")))
    worm_button = driver.find_element(By.XPATH, "//button[@data-name='Worms']")

    driver.execute_script("arguments[0].scrollIntoView({ block: 'center', inline: 'center', offsetTop: -200 });", worm_button)
    worm_button.click()
    time.sleep(0.5)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@class='actions-modal modal-in']")))
    driver.find_element(By.XPATH, "//div[@class='actions-modal-button']").click()

    time.sleep(0.5)
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//span[@class='modal-button modal-button-bold']")))
    driver.find_element(By.XPATH, "//span[@class='modal-button']").click()


buy_bait()

https://farmrpg.com/#!/store.php
Location https://farmrpg.com/#!/store.php not found in locations.csv
GOTO:  None -> town
//div[@class="view view-main navbar-through"]//a[@href='town.php']


In [203]:
def replenish_stamina():
    if driver.current_url != "https://farmrpg.com/#!/fishing.php?id=2":
        go_to_fish_location("farm pond")
    
    time.sleep(0.5)
    fish_divs = driver.find_elements(By.XPATH, "//img[@src='/img/items/fish.png']")

    def fish_coming_up(driver):
        for fish in fish_divs:
            if "catch" in fish.get_attribute("class"):
                return fish
    
    try:
        max_stamina = int(driver.find_element(By.XPATH, "//a[@href='supply.php']").get_attribute("innerHTML").split(" ")[0])
    except:
        print("Stamina is full!")
        return
        
    
    while True:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href,'changebait')]")))
        if int(driver.find_element(By.XPATH, "//a[contains(@href,'changebait.php')]/../strong").text) == 0:
            print("No bait left!")
            buy_bait()

        catchable = WebDriverWait(driver, 5).until(fish_coming_up)
        ActionChains(driver).move_to_element(catchable.find_element(By.XPATH, '..')).click().perform()

        time.sleep(0.2)

        try:
            WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'fishcaught finalcatch')]")))
            driver.find_element(By.XPATH, "//div[contains(@class,'fishcaught finalcatch')]").click()
            time.sleep(0.1)
        except:
            print("r")
            time.sleep(0.1)
            pass
            
        current_stamina = int(driver.find_element(By.XPATH, "//strong[@id='stamina']").get_attribute("innerHTML"))
        if current_stamina == max_stamina:
            break
        
replenish_stamina()

In [53]:
def fish_manually(target_location=None, num=3):
    
    if "fishing" not in driver.current_url:
        go_to_fish_location(target_location)

    time.sleep(0.5)
    fish_divs = driver.find_elements(By.XPATH, "//img[@src='/img/items/fish.png']")

    def fish_coming_up(driver):
        for fish in fish_divs:
            if "catch" in fish.get_attribute("class"):
                return fish
    
    time.sleep(1)

    for i in range(num):
        catchable = WebDriverWait(driver, 5).until(fish_coming_up)
        ActionChains(driver).move_to_element(catchable.find_element(By.XPATH, '..')).click().perform()

        time.sleep(0.2)

        try:
            WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'fishcaught finalcatch')]")))
            driver.find_element(By.XPATH, "//div[contains(@class,'fishcaught finalcatch')]").click()
            time.sleep(0.1)
        except:
            print("r")
            pass
        # ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "//div[contains(@class,'fishcaught')]")).click().perform()

fish_manually()


In [103]:
def get_craftable_items():
    goto("workshop")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'favcraftitems')]")))
    favourite_list = driver.find_elements(By.XPATH, "//div[contains(@class,'favcraftitems')]//li")
    favourite_inputs = driver.find_elements(By.XPATH, "//div[contains(@class,'favcraftitems')]//li//div[@class='item-title']//input")
    
    craftable_items = pd.DataFrame(
        {
            "name": [item.get_attribute("data-name") for item in favourite_list],
            "amount": [int(item.get_attribute("value")) for item in favourite_inputs]
        }
    )

    return craftable_items.sort_values("amount", ascending=False)

get_craftable_items()

https://farmrpg.com/#!/workshop.php


,name,amount
9,Shimmer Stone,51
0,Twine,33
11,Steel,33
4,Black Powder,30
14,Fancy Pipe,23
16,Lemonade,11
6,Board,9
3,Leather,4
5,Mushroom Paste,2
7,Wooden Plank,0


In [147]:
def craft_favourites():
    goto("workshop")

    i = 10

    while True:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'favcraftitems')]")))
        favourite_craft_buttons = driver.find_elements(
            By.XPATH, 
            "//div[contains(@class,'favcraftitems')]//li//div[@class='item-after']//button[contains(@class,'disable-select button btngreen craftbtnnc craftbtn')]"
        )
        if len(favourite_craft_buttons) == 0:
            break
        
        driver.execute_script("arguments[0].scrollIntoView({ block: 'center', inline: 'center', offsetTop: -200 });", favourite_craft_buttons[0])
        favourite_craft_buttons[0].click()
        time.sleep(1)

craft_favourites()

https://farmrpg.com/#!/workshop.php
Success!
Success!
Success!
Success!
Success!


In [179]:
def get_sellable_items():
    goto("town")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[@href='market.php']")))
    driver.find_element(By.XPATH, "//a[@href='market.php']").click()

    time.sleep(0.5)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[@class='sellallunlockedmaxbtn']")))
    sell_buttons = driver.find_elements(By.XPATH, "//button[contains(@class,'sellbtn')]")

    sellable_items = pd.DataFrame({
        "name": [item.get_attribute("data-name") for item in sell_buttons],
        "price": [int(item.find_element(By.XPATH, "./span").get_attribute("innerHTML").replace(",","")) for item in sell_buttons]
    })
    return sellable_items.sort_values("price", ascending=False)

get_sellable_items()

https://farmrpg.com/#!/market.php
Location https://farmrpg.com/#!/market.php not found in locations.csv
GOTO:  None -> town
//div[@class="view view-main navbar-through"]//a[@href='town.php']


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00BE72A3+45731]
	(No symbol) [0x00B72D51]
	(No symbol) [0x00A6880D]
	(No symbol) [0x00A9B940]
	(No symbol) [0x00A9BE0B]
	(No symbol) [0x00ACD1F2]
	(No symbol) [0x00AB8024]
	(No symbol) [0x00ACB7A2]
	(No symbol) [0x00AB7DD6]
	(No symbol) [0x00A931F6]
	(No symbol) [0x00A9439D]
	GetHandleVerifier [0x00EF0716+3229462]
	GetHandleVerifier [0x00F384C8+3523784]
	GetHandleVerifier [0x00F3214C+3498316]
	GetHandleVerifier [0x00C71680+611968]
	(No symbol) [0x00B7CCCC]
	(No symbol) [0x00B78DF8]
	(No symbol) [0x00B78F1D]
	(No symbol) [0x00B6B2C7]
	BaseThreadInitThunk [0x761BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D37C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77D37C3E+238]


In [178]:
sellable_items = ["sturdy shield","sturdy sword","fancy pipe"]

def sell_items(items=sellable_items, min_inventory=200):
    goto("town")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[@href='market.php']")))
    driver.find_element(By.XPATH, "//a[@href='market.php']").click()

    time.sleep(0.5)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[@class='sellallunlockedmaxbtn']")))
    sell_buttons = driver.find_elements(By.XPATH, "//button[contains(@class,'sellbtn')]")

    for sell_button in sell_buttons:
        if sell_button.get_attribute("data-name").lower() in items:
            driver.execute_script("arguments[0].scrollIntoView({ block: 'center', inline: 'center', offsetTop: -200 });", sell_button)
            sell_button.click()
            time.sleep(0.5)

            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@class='actions-modal modal-in']")))
            driver.find_element(By.XPATH, "//div[@class='actions-modal-button']").click()

            time.sleep(0.5)
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//span[@class='modal-button modal-button-bold']")))
            driver.find_element(By.XPATH, "//span[@class='modal-button modal-button-bold']").click()

sell_items()

https://farmrpg.com/#!/market.php
Location https://farmrpg.com/#!/market.php not found in locations.csv
GOTO:  None -> town
//div[@class="view view-main navbar-through"]//a[@href='town.php']


In [204]:
while True:
    if is_harvest_available():
        harvest()
    plant_crop("eggplant")
    
    explore(target_location="ember lagoon", num=100)
    craft_favourites()
    sell_items()

https://farmrpg.com/#!/workshop.php
GOTO:  workshop -> home
No Crops planted!
https://farmrpg.com/
GOTO:  home -> explore
//div[@class="view view-main navbar-through"]//a[@href='explore.php']
['Forest', 'Small Cave', 'Small Spring', 'Highland Hills', 'Cane Pole Ridge', 'Misty Forest', 'Black Rock Canyon', 'Mount Banon', 'Ember Lagoon']
No stamina left
https://farmrpg.com/#!/area.php?id=9
Location https://farmrpg.com/#!/area.php?id=9 not found in locations.csv
GOTO:  None -> fish
//div[@class="view view-main navbar-through"]//a[@href='fish.php']
r
r
r
https://farmrpg.com/#!/fishing.php?id=2
Location https://farmrpg.com/#!/fishing.php?id=2 not found in locations.csv
GOTO:  None -> explore
//div[@class="view view-main navbar-through"]//a[@href='explore.php']
['Forest', 'Small Cave', 'Small Spring', 'Highland Hills', 'Cane Pole Ridge', 'Misty Forest', 'Black Rock Canyon', 'Mount Banon', 'Ember Lagoon']
https://farmrpg.com/#!/area.php?id=9
Location https://farmrpg.com/#!/area.php?id=9 not f

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=119.0.6045.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00BE72A3+45731]
	(No symbol) [0x00B72D51]
	(No symbol) [0x00A6880D]
	(No symbol) [0x00A6C3EA]
	(No symbol) [0x00A6DB1A]
	(No symbol) [0x00A6DBC0]
	(No symbol) [0x00AA067C]
	(No symbol) [0x00A9F2AF]
	(No symbol) [0x00A9D89B]
	(No symbol) [0x00A9CBA1]
	(No symbol) [0x00A94FC0]
	(No symbol) [0x00AB7FDC]
	(No symbol) [0x00A94A4E]
	(No symbol) [0x00AB8254]
	(No symbol) [0x00ACB7A2]
	(No symbol) [0x00AB7DD6]
	(No symbol) [0x00A931F6]
	(No symbol) [0x00A9439D]
	GetHandleVerifier [0x00EF0716+3229462]
	GetHandleVerifier [0x00F384C8+3523784]
	GetHandleVerifier [0x00F3214C+3498316]
	GetHandleVerifier [0x00C71680+611968]
	(No symbol) [0x00B7CCCC]
	(No symbol) [0x00B78DF8]
	(No symbol) 

In [186]:
while True:
    harvest()
    plant_crop("carrot")
    ac = get_available_crops()
    if ac[ac["name"] == "Carrot Seeds"].iloc[0,1] < 100:
        pass
        # buy seeds
    time.sleep(121)


https://farmrpg.com/#!/explore.php
GOTO:  explore -> farm
//div[@class="view view-main navbar-through"]//a[contains(@href, 'xfarm.php?id')]
https://farmrpg.com/#!/xfarm.php?id=343807
https://farmrpg.com/#!/xfarm.php?id=343807
Planted Carrot Seeds
https://farmrpg.com/#!/xfarm.php?id=343807


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=119.0.6045.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00BE72A3+45731]
	(No symbol) [0x00B72D51]
	(No symbol) [0x00A6880D]
	(No symbol) [0x00A6C3EA]
	(No symbol) [0x00A6DD59]
	(No symbol) [0x00ACBFCC]
	(No symbol) [0x00AB7FDC]
	(No symbol) [0x00ACB7A2]
	(No symbol) [0x00AB7DD6]
	(No symbol) [0x00A931F6]
	(No symbol) [0x00A9439D]
	GetHandleVerifier [0x00EF0716+3229462]
	GetHandleVerifier [0x00F384C8+3523784]
	GetHandleVerifier [0x00F3214C+3498316]
	GetHandleVerifier [0x00C71680+611968]
	(No symbol) [0x00B7CCCC]
	(No symbol) [0x00B78DF8]
	(No symbol) [0x00B78F1D]
	(No symbol) [0x00B6B2C7]
	BaseThreadInitThunk [0x761BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D37C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77D37C3E+238]
